In [ ]:
input_file = "/kaggle/input/txtfile/requirements.txt"  # Replace with your file name
output_file = "/kaggle/working/install_requirements.txt"

with open(input_file, "r") as infile, open(output_file, "w") as outfile:
    for line in infile:
        line = line.strip()
        if line:  # Avoid processing empty lines
            outfile.write(f"pip install {line}\n")

print(f"Modified file saved as {output_file}")


Modified file saved as /kaggle/working/install_requirements.txt


In [28]:
!pip install datasets

In [29]:
!pip install einops

In [30]:
!pip install openai

In [31]:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [33]:
import torch
from torchvision import transforms
from PIL import Image

def load_image(image, input_size=448):
    if isinstance(image, Image.Image):
        # Convert RGBA images to RGB (discarding the alpha channel)
        if image.mode == 'RGBA':
            image = image.convert('RGB')

        # Define the transformation (resize, convert to tensor, and normalize)
        transform = transforms.Compose([
            transforms.Resize((input_size, input_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        # Apply the transformations to the image
        tensor = transform(image).unsqueeze(0)  # Add batch dimension

        # Clamp the values of the tensor to be within [0, 1]
        tensor = tensor.clamp(0, 1)

        # Move the tensor to the GPU (if needed) and convert to float32
        return tensor.to(torch.float32).cuda()  # Use float32 here
    else:
        raise ValueError("Input must be a PIL.Image object.")

In [44]:
import google.generativeai as genai
from PIL import Image
import base64
from openai import OpenAI
from transformers import PaliGemmaForConditionalGeneration, PaliGemmaProcessor
import torchvision.transforms as T
from transformers import AutoModel, AutoTokenizer
from transformers import AutoProcessor, LlavaForConditionalGeneration



# Configure the API key
GOOGLE_API_KEY = "KEEP API KEY HEre"
genai.configure(api_key=GOOGLE_API_KEY)

def call_gemini_1_5_pro(text_query, image, temperature=0.4):
    """Call Gemini 1.5 Pro Vision model with text and image input."""
    model = genai.GenerativeModel('gemini-1.5-pro')

    # Open the image

    try:
        # Generate content
        response = model.generate_content(
            [text_query, image],
            generation_config=genai.types.GenerationConfig(
                candidate_count=1,
                max_output_tokens=1024,
                temperature=temperature
            )
        )
        # Return the generated response
        return response.candidates[0].content.parts[0].text
    except Exception as e:
        print(f"Error during Gemini 1.5 Pro Vision call: {e}")
        return ""

def call_gemini_1_5_flash(text_query, image, temperature=0.4):
    """Call Gemini 1.5 Flash Vision model with text and image input."""
    model = genai.GenerativeModel('gemini-1.5-flash')

    # Open the image


    try:
        # Generate content
        response = model.generate_content(
            [text_query, image],
            generation_config=genai.types.GenerationConfig(
                candidate_count=1,
                max_output_tokens=1024,
                temperature=temperature
            )
        )
        # Return the generated response
        return response.candidates[0].content.parts[0].text
    except Exception as e:
        print(f"Error during Gemini 1.5 Flash Vision call: {e}")
        return ""


def call_gpt_vision(text_query, image_path,temperature=0.4):

    # Initialize the client with API key
    openai.api_key = os.environ.get("OPENAI_API_KEY")

    # Get the base64-encoded image
    base64_image = encode_image(image_path)

    # Construct the message with both the text query and image
    messages = [
        {
            "role": "user",
            "content": f"{text_query}\nBelow is the image I want you to analyze:",
        },
        {
            "role": "user",
            "content": f"![image](data:image/jpeg;base64,{base64_image})",
        },
    ]

    # Send the chat completion request
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages,
        max_tokens=300
    )

    return response['choices'][0]['message']['content']

In [36]:
def call_internvl2_vision(text_query, image):
    # Model and Tokenizer initialization
    path = 'OpenGVLab/InternVL2_5-1B'
    model = AutoModel.from_pretrained(
        path,
        torch_dtype=torch.bfloat16,
        low_cpu_mem_usage=True,
        use_flash_attn=True,
        trust_remote_code=True
    ).eval().cuda()

    tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)

    # Load and preprocess the image
    pixel_values = load_image(image, input_size=448).to(torch.bfloat16).cuda()

    # Set generation configuration
    generation_config = dict(max_new_tokens=512, do_sample=True)

    # Create question based on text_query
    question = f'<image>\n{text_query}\n'

    # Perform inference
    response = model.chat(tokenizer, pixel_values, question, generation_config)

    # Return the model's response
    return response

In [37]:
import os
import torch
import numpy as np
import pandas as pd
import json
from tqdm import tqdm

# Function definitions remain the same
def call_vision(text_query, image_path, model):
    if model == "gemini_1":
        response = call_gemini_1_5_flash(text_query, image_path)
    elif model == "gemini_2":
        response = call_gemini_1_5_pro(text_query, image_path)
    elif model == "gpt4":
        respone = call_gpt_vision(text_query, image_path)
    elif model == "intern_vl2":
        response =  call_internvl2_vision(text_query, image_path)
    else:
        raise NotImplementedError
    return response


In [38]:
def prepare_vision_prompt(dataset):

    prompts = []

    for row in dataset:

        image, well_edge = row["image"], row["well_edge"]

        question = "Does the below Image contains elements of a microtiter plate well edge??"

        prompt = f"""Given the image of a cell , answer the following mutliple choice question.


        Question: {question}

        choices:
        1)True
        1)False

        select the option and no need of explanation.

        """

        prompts.append({
            "image": image,
            "true_label": well_edge,
            "query": prompt
        })

    return prompts

def check_correctness(vision_response, true_label):
    # Extract the relevant part of the response (e.g., "False" from "2) False")
    # If the model's response is in a format like "2) False", split by the ")"
    response = vision_response.split(')')[-1].strip()  # Extract the part after ")"

    # Check if the response matches the true label
    if response == true_label:
        return "True"
    else:
        return "False"

In [47]:
# Model and options setup (replace argparse with direct variables)
model = "intern_vl2"  # Specify the model to use

result_dir = "micronscopic1_benchmark"

if not os.path.exists(result_dir):
    os.makedirs(result_dir)

In [48]:
# Load the dataset
from datasets import load_dataset

dataset = load_dataset("mario-dg/brightfield-microscopy-scc",split="validation")

# change the ids of dataset to have only what comes after the last '/'
dataset = dataset.map(
    lambda x: {
        "image": x["image"],  # Retain the image data as is
        "label": x["label"],  # Extract the label
        "width": x["width"],  # Extract image width
        "height": x["height"],  # Extract image height
        "objects": x["objects"],  # Extract object annotations
        "well_edge": x["well_edge"],  # Extract well edge information
    }
)

dataset = dataset.select(range(10))

# Prepare prompts
prompts = prepare_vision_prompt(dataset)

results = []

# Initialize results DataFrame
for prompt_data in tqdm(prompts, desc="Evaluating Prompts"):
    # Extract prompt details
    image = prompt_data["image"]
    true_label = prompt_data["true_label"]
    prompt= prompt_data["query"]

    # Call the vision model
    vision_response = (
        call_vision(prompt, image, model)
        if model in ['gemini_1', 'gemini_2', 'gpt4','intern_vl2']
        else ""
    )

    print(f"True Label: {true_label}\n Vision Response: {vision_response}")


    # Append results
    results.append({
        "prompt" : prompt,
        "response": {"vision": vision_response},
        "true_label": true_label
    })

# Convert results to a DataFrame
df_results = pd.DataFrame(results)

# Save results to a JSON file
result_path = f"{result_dir}/{model}_Microscopic_ObjectDetection_results.json"
df_results.to_json(result_path, orient="records", indent=4)



Evaluating Prompts:  10%|█         | 1/10 [00:04<00:38,  4.32s/it]

True Label: True
 Vision Response: Options could be True or False.

1) True
2) False

Options could be True or False.

1) True
2) False


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Evaluating Prompts:  20%|██        | 2/10 [00:07<00:29,  3.71s/it]

True Label: True
 Vision Response: False


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Evaluating Prompts:  30%|███       | 3/10 [00:10<00:24,  3.56s/it]

...

True Label: True
 Vision Response: 1)True


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Evaluating Prompts:  90%|█████████ | 9/10 [00:31<00:03,  3.43s/it]

True Label: False
 Vision Response: 1)True


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Evaluating Prompts: 100%|██████████| 10/10 [00:34<00:00,  3.49s/it]

True Label: True
 Vision Response: 1) True
